In [2]:
import pandas as pd

In [3]:
mails = pd.read_csv('spam.csv',engine='python')

In [4]:
mails.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
for col in mails.columns:
    print(col)

v1
v2
Unnamed: 2
Unnamed: 3
Unnamed: 4


In [6]:
mails=mails.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)

In [7]:
mails.rename(columns={'v1':'category','v2':'description'}, inplace=True)

In [8]:
mails.describe(include='all')

,category,description
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [9]:
mails.shape

(5572, 2)

In [10]:
mails.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
category       5572 non-null object
description    5572 non-null object
dtypes: object(2)
memory usage: 87.1+ KB


In [11]:
mails['category'].unique()

array(['ham', 'spam'], dtype=object)

In [12]:
len(mails['category'].unique())

2

In [13]:
import seaborn as sns
sns.countplot(data=mails, x='category')

In [14]:
mails['spam'] = mails['category'].apply(lambda x: 1 if x=='spam' else 0)

In [15]:
mails.head()

,category,description,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [53]:
X=mails['description']
y=mails['spam']

In [17]:
from sklearn.model_selection import train_test_split as tts
X_train,X_test,y_train,y_test = tts(X,y,test_size=0.2)

In [18]:
from sklearn.ensemble import RandomForestClassifier
rfmodel = RandomForestClassifier(n_estimators=50,oob_score=True,n_jobs=-1,random_state=20)

In [19]:
#Gives error for Random forest Classifier due to X being string
rfmodel.fit(X_train,y_train)

ValueError: could not convert string to float: "Just trying to figure out when I'm suppose to see a couple different people this week. We said we'd get together but I didn't set dates"

In [22]:
#Doing NLP for text description
mails['description']=mails['description'].str.lower()

In [23]:
mails['description']=mails['description'].str.replace('http\S+|www.\S+',' ',case=False)

In [24]:
mails['description']=mails['description'].str.replace('\d+',' ')

In [25]:
mails['description']=mails['description'].str.replace('\W',' ')

In [26]:
mails['description']=mails['description'].str.replace(' +',' ')
mails['description']=mails['description'].str.replace('\s{2,}',' ')

In [27]:
#Lammatization using textblob
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from textblob import TextBlob, Word

def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', 
                "N": 'n',
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]  
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

[nltk_data] Downloading package punkt to C:\Users\mathan
[nltk_data]     mohan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mathan mohan\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to C:\Users\mathan
[nltk_data]     mohan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [28]:
mails['description']=mails['description'].apply(lemmatize_with_postag)

In [29]:
#Getting stop words from nltk
stop_words_ls = nltk.corpus.stopwords.words('english')

In [30]:
#Removing stop words
for word in stop_words_ls:
    mails['description']=mails['description'].replace(to_replace=r'\b%s\b'%word, value="", regex=True)

In [31]:
mails['description']=mails['description'].map(lambda x:x.strip())

In [32]:
def pos(text):
    return TextBlob(text).tags

In [33]:
pos_out=mails['description'].apply(pos)

In [34]:
pos_out=pd.DataFrame(pos_out)

In [35]:
#Converted into dataframe
pos_out

,description
0,"[(go, VB), (jurong, JJ), (point, NN), (crazy, ..."
1,"[(ok, JJ), (lar, JJ), (joking, NN), (wif, NN),..."
2,"[(free, JJ), (entry, NN), (wkly, VBD), (comp, ..."
3,"[(u, JJ), (dun, NNS), (say, VBP), (early, JJ),..."
4,"[(nah, RB), (think, VBP), (go, VB), (usf, JJ),..."
5,"[(freemsg, NN), (hey, NN), (darling, VBG), (we..."
6,"[(even, RB), (brother, RBR), (like, IN), (spea..."
7,"[(per, IN), (request, NN), (melle, NN), (melle..."
8,"[(winner, NN), (value, NN), (network, NN), (cu..."
9,"[(mobile, JJ), (month, NN), (u, JJ), (r, NN), ..."


In [36]:
mails['description']=pos_out['description'].map(lambda x:" ".join(["/".join(x) for x in x]))

In [74]:
#Now splitting test & train sets
X_train,X_test,y_train,y_test = tts(X,y,test_size=0.2,random_state=101)

In [75]:
#Apply tdidf vectorization for string data
from sklearn.feature_extraction.text import TfidfVectorizer
tf_vect = TfidfVectorizer(lowercase=True, use_idf=True, smooth_idf=True, sublinear_tf=False,ngram_range=(2,2))

In [76]:
#Transform data into vectors
X_train_tf = tf_vect.fit_transform(X_train)
X_train_tf

<4457x19164 sparse matrix of type '<class 'numpy.float64'>'
	with 71732 stored elements in Compressed Sparse Row format>

In [77]:
#Fitting into random forest model
rfmodel.fit(X_train_tf,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
                       oob_score=True, random_state=20, verbose=0,
                       warm_start=False)

In [78]:
oob = rfmodel.oob_score_
print("OOB score: ",oob)

OOB score:  0.9699349338119811


In [79]:
#Transforming test set into vectors
X_test_tf = tf_vect.transform(X_test)

In [80]:
#Training set shape
X_train_tf.shape

(4457, 19164)

In [81]:
#Test set shape
X_test_tf.shape

(1115, 19164)

In [61]:
X_test_tf

<1115x2685 sparse matrix of type '<class 'numpy.float64'>'
	with 12961 stored elements in Compressed Sparse Row format>

In [82]:
#predicting with test data set
y_pred = rfmodel.predict(X_test_tf)

In [84]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,y_pred))

[[977   0]
 [ 32 106]]


In [91]:
from sklearn.metrics import accuracy_score,recall_score,precision_score
print("Recall: ",recall_score(y_test,y_pred))
print("Accuracy: ",accuracy_score(y_test, y_pred))
print("Precision: ",precision_score(y_test, y_pred))

Recall:  0.7681159420289855
Accuracy:  0.9713004484304932
Precision:  1.0


In [92]:
#confusion matrix is one of the methods used to check the accuracy of a classification model.
pd.crosstab(y_test,y_pred,rownames=['Actual Result'],colnames=['Predicted Result'])

Predicted Result,0,1
Actual Result,,
0,977,0
1,32,106


In [93]:
# importance of all the features in this dataset
list(zip(X_train_tf,rfmodel.feature_importances_))

[(<1x19164 sparse matrix of type '<class 'numpy.float64'>'
  	with 7 stored elements in Compressed Sparse Row format>, 0.0),
 (<1x19164 sparse matrix of type '<class 'numpy.float64'>'
  	with 118 stored elements in Compressed Sparse Row format>, 0.0),
 (<1x19164 sparse matrix of type '<class 'numpy.float64'>'
  	with 5 stored elements in Compressed Sparse Row format>, 0.0),
 (<1x19164 sparse matrix of type '<class 'numpy.float64'>'
  	with 5 stored elements in Compressed Sparse Row format>, 0.0),
 (<1x19164 sparse matrix of type '<class 'numpy.float64'>'
  	with 31 stored elements in Compressed Sparse Row format>, 0.0),
 (<1x19164 sparse matrix of type '<class 'numpy.float64'>'
  	with 6 stored elements in Compressed Sparse Row format>,
  1.544691312305425e-09),
 (<1x19164 sparse matrix of type '<class 'numpy.float64'>'
  	with 19 stored elements in Compressed Sparse Row format>, 0.0),
 (<1x19164 sparse matrix of type '<class 'numpy.float64'>'
  	with 19 stored elements in Compressed S

In [96]:
from sklearn.metrics import classification_report
target_names=['spam','not-spam']
print(classification_report(y_test,y_pred,target_names=target_names))

              precision    recall  f1-score   support

        spam       0.97      1.00      0.98       977
    not-spam       1.00      0.77      0.87       138

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.93      1115
weighted avg       0.97      0.97      0.97      1115



In [97]:
#Apply Naive Bayes Multinomial
from sklearn.naive_bayes import MultinomialNB
multi_nom_model = MultinomialNB()

In [98]:
multi_nom_model.fit(X_train_tf,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [100]:
pred_naiv = multi_nom_model.predict(X_test_tf)

In [102]:
print(accuracy_score(y_test,pred_naiv))

0.9650224215246637


In [104]:
print(classification_report(pred_naiv, y_test))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98      1014
           1       0.72      0.99      0.84       101

    accuracy                           0.97      1115
   macro avg       0.86      0.98      0.91      1115
weighted avg       0.97      0.97      0.97      1115



In [105]:
#Apply Naive Bayes Binomial or Bernoulli
from sklearn.naive_bayes import BernoulliNB
bernou_model = BernoulliNB(binarize=0.0)

In [106]:
bernou_model.fit(X_train_tf,y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [107]:
bernou_model.score(X_test_tf,y_test)

0.9659192825112107

In [111]:
bernou_model.predict(X_test_tf)

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)